In [105]:
# Import packages and load the data
import numpy as np
import numpy.random as rnd
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
cmap1 = matplotlib.cm.get_cmap('summer')
# This next command is specifically for Jupyter Notebook
%matplotlib notebook

In [106]:
# Load the data as a NumPy array
data = np.loadtxt('NewMacroSeries.txt', delimiter = ',')
#We rename the each of the columns of the data with variables corresponding to the model
#(c:consumption, k:capital, w:wage, r=interest rate)
c_data = data[:,0]
k_data = data[:,1]
w_data = data[:,2]
r_data = data[:,3]
y_data = data[:,4]

#Parameter assumptions
beta = 0.99
k_1 = k_data.mean()

#Simulation assumptions (for the uniform matrix)
S = 1000 
T = 100

Part a)

In [107]:
#Matrix of S = 1; 000 simulations (columns) of T = 100 (rows) from a uniform
simulations = np.random.uniform(0,1,(T,S))

In [108]:
# We need to simulate the variables according with the equations given in the pdf

def variables_simulated(alpha, beta, rho, mu, sigma, simulations):
    epsilon = sts.norm.ppf(simulations,0,sigma)
    z = np.zeros((T,S))  
    z_t_1 = mu
    for i in range(100):   
        z_t = rho*z_t_1 + (1-rho)*mu + epsilon[i,:]
        z[i,:] = z_t
        z_t_1 = z_t
    
    k = np.zeros((T,S))
    k_lag = np.zeros((T,S))
    k_t_1 = k_1
    for j in range(100):
        k_t = alpha*beta*np.exp(z[j,:])*k_t_1**alpha
        k_lag[j,:] = k_t_1
        k[j,:] = k_t
        k_t_1 = k_t

    w = (1-alpha)*np.exp(z)*k_lag**alpha
    r = alpha*np.exp(z)*k_lag**(alpha-1)
    c = w + r*k_lag-k
    y = np.exp(z)*k_lag**alpha
    
    return c,k_lag,k,w,r,y

In [109]:
# We compute de moment from the data 

moment_data1 = c_data.mean()
moment_data2 = k_data.mean()
moment_data3 = (c_data/y_data).mean()
moment_data4 = y_data.var()
moment_data5= np.corrcoef(c_data[0:99],c_data[1:100])[0,1]
moment_data6= np.corrcoef(c_data, k_data)[0,1]
moment_data=np.array([moment_data1, moment_data2, moment_data3, moment_data4, moment_data5, moment_data6])

In [110]:
#Taking our simulated data we compute the simulated moments given in the pdf
def moments_simulated(alpha, beta, rho, mu, sigma, simulations):
    c,k_lag,k,w,r,y = variables_simulated(alpha, beta, rho, mu, sigma, simulations)
    moment_simul1 = np.mean(c)
    moment_simul2 = np.mean(k)
    moment_simul3 = np.mean(c/y)
    moment_simul4 = np.mean(np.var(y,axis=0)) 
    correl_c = np.zeros((1,S))
    correl_ck = np.zeros((1,S))
    
    for i in range(1000):   
        correl_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
        correl_ck[0,i] = np.corrcoef(c[:,i],k_lag[:,i])[0,1]
    
    moment_simul5 = correl_c.mean()
    moment_simul6 = correl_ck.mean()
    moment_simul = np.array([moment_simul1,moment_simul2,moment_simul3,moment_simul4,moment_simul5,moment_simul6])
    return moment_simul

In [111]:
#We define the error error and criterion function

def err_vec(alpha, beta, rho, mu, sigma, simulations):
    moment_simul = moments_simulated(alpha, beta, rho, mu, sigma, simulations)
    return (moment_simul - moment_data)/moment_data

def criterion_SMM(params, *args):
    alpha, rho, mu, sigma = params
    beta, simulations = args
    err = err_vec(alpha, beta, rho, mu, sigma, simulations)
    W = np.eye(6)
    return err @ W @ err.T

In [112]:
# We guess some initial values for the parameters (I am taking the values that I got from the MLE estimation of PS2)
alpha_init=0.7018970334618823
rho_init=0.4800169691930428
mu_init=5.077378123134153
sigma_init=0.2

params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
bounds = ((0.01,0.99),(-0.99,0.99),(5,14),(0.01,1.1))
smm_args = (beta,simulations)

#Minimization problem for the SMM estimation
result = opt.minimize(criterion_SMM, params_init, args = (smm_args), method = 'L-BFGS-B', bounds = bounds)
alpha_smm, rho_smm, mu_smm, sigma_smm = result.x

print('alpha SMM = ', alpha_smm)
print('rho SMM = ', rho_smm)
print('mu SMM = ', mu_smm)
print('sigma SMM = ', sigma_smm)

print('Error vec =', err_vec(alpha_smm, 0.99, rho_smm, mu_smm, sigma_smm, simulations))
print("Criterion function value =", criterion_SMM(result.x, 0.99, simulations))

alpha SMM =  0.42103423036100984
rho SMM =  0.9174627639046471
mu SMM =  9.92760014039361
sigma SMM =  0.09004838136874012
Error vec = [ 7.44100555e-04 -7.38807276e-04 -1.75263276e-03 -1.98616315e-06
  3.30035526e-04 -3.10640056e-04]
Criterion function value = 4.376668049896011e-06


In [113]:
#We compute the Jacobian matrix (needed for the estimation of the standard errors)

def Jac_err2(data_vals, unif_vals, alpha, rho, mu, sigma, simple=False):

    Jac_err = np.zeros((6, 4))
    h_alpha = 1e-4 * alpha
    h_rho = 1e-4 * rho
    h_mu = 1e-4 * mu
    h_sigma = 1e-4 * sigma
    Jac_err[:, 0] = \
        ((err_vec(alpha + h_alpha, 0.99, rho, mu, sigma, simulations) -
          err_vec(alpha - h_alpha, 0.99, rho, mu, sigma, simulations)) / (2 * h_alpha)).flatten()
    Jac_err[:, 1] = \
        ((err_vec(alpha, 0.99, rho + h_rho, mu, sigma, simulations) -
          err_vec(alpha, 0.99, rho - h_rho, mu, sigma, simulations)) / (2 * h_rho)).flatten()
    Jac_err[:, 2] = \
        ((err_vec(alpha, 0.99, rho, mu + h_mu, sigma, simulations) -
          err_vec(alpha, 0.99, rho, mu - h_mu, sigma, simulations)) / (2 * h_mu)).flatten()
    Jac_err[:, 3] = \
        ((err_vec(alpha, 0.99, rho, mu, sigma + h_sigma, simulations) -
          err_vec(alpha, 0.99, rho, mu, sigma - h_sigma, simulations)) / (2 * h_sigma)).flatten()
    
    return Jac_err

In [114]:
#Standard errors
SS = simulations.shape[1]
d_err2 = Jac_err2(data, simulations, alpha_smm, rho_smm, mu_smm, sigma_smm, False)
print(d_err2)
W = np.eye(6)
print(W)
SigHat2 = (1 / SS) * lin.inv(d_err2.T @ W @ d_err2)
print(SigHat2)
print('Std. err. alpha=', np.sqrt(SigHat2[0, 0]))
print('Std. err. rho=', np.sqrt(SigHat2[1, 1]))
print('Std. err. mu=', np.sqrt(SigHat2[2, 2]))
print('Std. err. sigma=', np.sqrt(SigHat2[3, 3]))

[[ 2.70593112e+01  8.39184781e-01  1.71643424e+00  1.54452317e+00]
 [ 3.10889132e+01  8.37941273e-01  1.71389082e+00  1.54223448e+00]
 [-1.69462513e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.13259563e+01  1.19087518e+01  3.45979140e+00  2.71079197e+01]
 [ 1.41334352e-01  4.23812633e-01  7.97041479e-04 -8.41316085e-02]
 [ 2.42068188e-01  4.29312671e-01  6.29637902e-03 -8.10755398e-02]]
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[ 8.95633616e-05 -2.74077236e-05 -1.50622633e-03  1.66231471e-06]
 [-2.74077236e-05  2.35763969e-03  2.05581471e-04 -9.99680539e-04]
 [-1.50622633e-03  2.05581471e-04  2.55769400e-02  5.14002059e-05]
 [ 1.66231471e-06 -9.99680539e-04  5.14002059e-05  4.30263418e-04]]
Std. err. alpha= 0.009463792137082695
Std. err. rho= 0.0485555320111543
Std. err. mu= 0.15992792141663875
Std. err. sigma= 0.020742791956517733


Part b:

In [115]:
def get_Err_mat(unif_vals, alpha, beta, rho, mu, sigma, simple=False):
  
    R = 6
    SS = simulations.shape[1]
    Err_mat = np.zeros((R, SS))
    mm_simulated1, mm_simulated2, mm_simulated3, mm_simulated4, mm_simulated5, mm_simulated6 = moments_simulated(alpha, beta, rho, mu, sigma, simulations)
    if simple:
        Err_mat[0, :] = mm_simulated1 - moment_data1
        Err_mat[1, :] = mm_simulated2 - moment_data2
        Err_mat[2, :] = mm_simulated3 - moment_data3
        Err_mat[3, :] = mm_simulated4 - moment_data4
        Err_mat[4, :] = mm_simulated5 - moment_data5
        Err_mat[5, :] = mm_simulated6 - moment_data6
    else:
        Err_mat[0, :] = (mm_simulated1 - moment_data1) / moment_data1
        Err_mat[1, :] = (mm_simulated2 - moment_data2) / moment_data2
        Err_mat[2, :] = (mm_simulated3 - moment_data3) / moment_data3
        Err_mat[3, :] = (mm_simulated4 - moment_data4) / moment_data4
        Err_mat[4, :] = (mm_simulated5 - moment_data5) / moment_data5
        Err_mat[5, :] = (mm_simulated6 - moment_data6) / moment_data6
        
    return Err_mat

In [121]:
Err_mat = get_Err_mat(simulations, alpha_smm, 0.99, rho_smm, mu_smm, sigma_smm, False)
VCV = (1 / SS) * (Err_mat @ Err_mat.T)
print(VCV)
W_hat = lin.inv(VCV)
print(W_hat)

[[ 5.53685636e-07 -5.49746904e-07 -1.30413501e-06 -1.47790510e-09
   2.45579618e-07 -2.31147438e-07]
 [-5.49746904e-07  5.45836191e-07  1.29485783e-06  1.46739179e-09
  -2.43832648e-07  2.29503134e-07]
 [-1.30413501e-06  1.29485783e-06  3.07172159e-06  3.48101460e-09
  -5.78431074e-07  5.44437939e-07]
 [-1.47790510e-09  1.46739179e-09  3.48101460e-09  3.94484406e-12
  -6.55504400e-10  6.16981833e-10]
 [ 2.45579618e-07 -2.43832648e-07 -5.78431074e-07 -6.55504400e-10
   1.08923448e-07 -1.02522254e-07]
 [-2.31147438e-07  2.29503134e-07  5.44437939e-07  6.16981833e-10
  -1.02522254e-07  9.64972446e-08]]
[[-1.75341996e+20  4.11593158e+20 -2.39340238e+20 -2.71225316e+23
  -2.47632818e+21 -9.45348469e+20]
 [ 4.10658167e+20 -5.77981766e+20  4.71181206e+20  2.55673829e+23
   2.46988660e+21  6.89289321e+20]
 [-2.38487100e+20  4.70454827e+20 -2.93165236e+20 -1.71051199e+23
  -1.73559147e+21 -7.86421729e+20]
 [-2.72485362e+23  2.60077905e+23 -1.74090814e+23 -1.95241917e+26
  -1.35174616e+24 -4.768

In [122]:
def criterion2_SMM(params, *args):
    alpha, rho, mu, sigma = params
    beta, simulations, W_hat = args
    err = err_vec(alpha, beta, rho, mu, sigma, simulations)
    return err @ W_hat @ err.T

params_init2 = np.array([alpha_smm, rho_smm, mu_smm, sigma_smm])
bounds2 = ((0.01,0.99),(-0.99,0.99),(5,14),(0.01,1.1))
W_hat = lin.inv(VCV)
smm_args2 = (beta,simulations,W_hat)

results2 = opt.minimize(criterion2_SMM, params_init2, args = (smm_args2), method = 'L-BFGS-B', bounds = bounds2)
alpha_smm2, rho_smm2, mu_smm2, sigma_smm2 = results2.x

#I don't know what is the problem

print('alpha SMM2 = ', alpha_smm2)
print('rho SMM2 = ', rho_smm2)
print('mu SMM2 = ', mu_smm2)
print('sigma SMM2 = ', sigma_smm2)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: overflow encountered in multiply
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in multiply
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in multiply
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in multiply
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in subtract
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in multiply


alpha SMM2 =  0.42103423036100984
rho SMM2 =  0.9174627639046471
mu SMM2 =  9.92760014039361
sigma SMM2 =  0.09004838136874012


In [120]:
#Standard errors
SS = simulations.shape[1]
d_err2_2 = Jac_err2(data, simulations, alpha_smm, rho_smm, mu_smm, sigma_smm, False)
print(d_err2_2)
W = W_hat
print(W)
SigHat2_2 = (1 / SS) * lin.inv(d_err2_2.T @ W_hat @ d_err2_2)

#I don't know what is the problem

print(SigHat2_2)
print('Std. err. alpha_2=', np.sqrt(SigHat2_2[0, 0]))
print('Std. err. rho_2=', np.sqrt(SigHat2_2[1, 1]))
print('Std. err. mu_2=', np.sqrt(SigHat2_2[2, 2]))
print('Std. err. sigma_2=', np.sqrt(SigHat2_2[3, 3]))

[[ 2.70593112e+01  8.39184781e-01  1.71643424e+00  1.54452317e+00]
 [ 3.10889132e+01  8.37941273e-01  1.71389082e+00  1.54223448e+00]
 [-1.69462513e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.13259563e+01  1.19087518e+01  3.45979140e+00  2.71079197e+01]
 [ 1.41334352e-01  4.23812633e-01  7.97041479e-04 -8.41316085e-02]
 [ 2.42068188e-01  4.29312671e-01  6.29637902e-03 -8.10755398e-02]]
[[-1.75341996e+20  4.11593158e+20 -2.39340238e+20 -2.71225316e+23
  -2.47632818e+21 -9.45348469e+20]
 [ 4.10658167e+20 -5.77981766e+20  4.71181206e+20  2.55673829e+23
   2.46988660e+21  6.89289321e+20]
 [-2.38487100e+20  4.70454827e+20 -2.93165236e+20 -1.71051199e+23
  -1.73559147e+21 -7.86421729e+20]
 [-2.72485362e+23  2.60077905e+23 -1.74090814e+23 -1.95241917e+26
  -1.35174616e+24 -4.76849797e+23]
 [-2.47886825e+21  2.48905382e+21 -1.75110737e+21 -1.33787979e+24
  -1.35479291e+22 -7.81760273e+21]
 [-9.42580360e+20  6.87832499e+20 -7.87242941e+20 -4.65774508e+23
  -7.76481452e+21 -4.72367

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in sqrt
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':
